# LL Resummation, starting from a dipole

If you use the package ngl\_resum, please cite [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660).

In this example, we will use the ngl\_resum package to resum the non-global logarithms of a rapidity gap, starting from a single color dipole along the beam axis. We suggest this notebook to be used in Binder:
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/MarcelBalsiger/ngl_resum/master?filepath=%2Fdocs%2Fresummator_dipole.ipynb) 

To have this example working as a jupyter notebook, one needs to have the packages numpy, physt and - obviously - ngl\_resum installed. The easiest way to do this is to use <code>pip install numpy physt ngl_resum</code>. Details may be found here: [https://packaging.python.org/tutorials/installing-packages/#use-pip-for-installing](https://packaging.python.org/tutorials/installing-packages/#use-pip-for-installing)

Lastly, one can also download the file resummator\_dipole.py from [https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples](https://github.com/MarcelBalsiger/ngl_resum/tree/master/examples) and play around with it, using this documentation on the side to explain the important steps.

### Imports

We start by importing the packages we need. In the file resummator\_dipole.py, we will also import argparse, but to streamline this documentation, we will not discuss the self-explanatory parsing of variables.

In [1]:
import time
import numpy as np
import ngl_resum as ngl

### Constants

We will now define some constants that will be used throughout the example to set up the shower. In resummator\_dipole.py, these constants are given by passing arguments from the command line.

Our end result of the resummation will be a histogram from t=0 to t=<code>tmax</code> consisting of <code>nbins</code> bins:

In [2]:
nbins = 10
tmax  = 0.1

We will shower the dipole <code>nsh</code> times:

In [3]:
nsh = 1000

We have to apply a cutoff of collinear emissions to massless legs. The parameter of <code>showerCutoff</code> corresponds to $\eta_{max}$ as discussed in (A.14) of [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660), reliable values are between 4 and 7. This cutoff is left to be defined by the user to speed up the calculation of test codes (smaller values), as computational time rises significantly with this cutoff.

In [4]:
showerCutoff = 6.

### Building the Event

We will now set up the event to eventually shower and choose this to be a back-to-back dipole along the beam axis with both legs of velocity <code>b</code>. A dipole consists of two legs, each represented by a <code>FourVector</code>, which is one of the classes of ngl\_resum. The two legs, normed to be of energy 1, are stored in an array:

In [5]:
b = 0.9
dipole = [ngl.FourVector(1,0,0,b),ngl.FourVector(1,0,0,-b)]

Now to instantiate the <code>Event</code> to shower, we just have to pass this <code>dipole</code> into the <code>feedDipole</code>-parameter:

In [6]:
ev = ngl.Event(feedDipole=dipole)

Our <code>Event</code> is prepared and almost ready to shower. All that's left to do now is to define the region of soft energy where we veto energetic radiation.

### Defining the outside region

We want to define the outside region to be a rapidity gap as depicted in the picture: 

<img src="./figures/outside_dipole.jpg" width=380px>

We have to first instantiate an <code>OutsideRegion</code>:

In [7]:
outsideRegion = ngl.OutsideRegion()

This does not do much yet. The <code>OutsideRegion</code> class is mainly a stub, consisting of one single method, <code>outside</code>. This method does not do anything without the user actually telling it what to do. The user has to code it in such a way that it will return either <code>True</code> or <code>False</code>, corresponding to whether the <code>FourVector</code> in the argument lands in the outside region or not. So, to set up our instance of <code>OutsideRegion</code> properly, we first write a function as follows:

In [8]:
def _outside(self,v):
    rapRangeMax = 0.8
    rapRangeMin = 0.0
    return (abs(v.rap)<rapRangeMax) and (abs(v.rap)>=rapRangeMin)

Note that the function needs to have exactly these two arguments, <code>self</code> and a <code>FourVector</code>. The argument <code>self</code> contains the instantiated <code>OutsideRegion</code> class, the use of which is not part of this example and is explained thoroughly in the example resummator\_LHE.py. The <code>FourVector</code> we feed into the method is the fourvector to be tested if it is outside.

Now, we need to connect <code>outsideRegion</code> (our instance of the <code>OutsideRegion</code> class) with the function <code>_outside(self,v)</code> we have just coded above. We do this by invoking the following code:

In [9]:
outsideRegion.outside = _outside.__get__(outsideRegion,ngl.OutsideRegion)

This command takes the stub of <code>outside(self, v)</code> from our instance <code>outsideRegion</code> and replaces it with the function <code>_outside(self, v)</code> from above. Now, our <code>outsideRegion</code> is properly set up and its <code>outside(self,v)</code>-method returns True, if <code>v</code> lands in the red region of the picture from above. To illustrate its behaviour, let us check two FourVectors, one that is outside and one that is inside: 

In [10]:
fvOut = ngl.FourVector(1,0,1,0) # light-like FourVector along y-axis - is outside!
fvIn = ngl.FourVector(1,0,0,1)  # light-like FourVector along z-axis - is inside!

In [11]:
outsideRegion.outside(fvOut)

True

In [12]:
outsideRegion.outside(fvIn)

False

We have now collected all the required ingredients of our shower and we resum the non-global logarithms of this specific problem.

### Showering the dipole

To set up the showering procedure of this problem, we now need to instantiate the <code>Shower</code> class of  ngl\_resum with all the required ingrdients:

In [13]:
shower = ngl.Shower(ev,outsideRegion,nsh,nbins,tmax,showerCutoff)

That's it! Now what we are left to do is to actually shower the event. To do this, we just have to invoke the <code>shower()</code>-method of our instance of the Shower class. To keep track of the required time for testing purposes, we also measure the runtime:

In [14]:
timeStart = time.time()
shower.shower()    
print('runtime=', time.time()-timeStart,' sec')    

runtime= 3.6651175022125244  sec


Done!

### Display results

The last thing we need to do is print the results of the resummation. To do this, we will can simply print out the <code>Hist</code> of <code>shower.resLL</code>:

In [15]:
shower.resLL

    t   | entries  |  error  
--------|----------|---------
 0.0050 | 0.831636 | 0.059415
 0.0150 | 0.799672 | 0.058100
 0.0250 | 0.682622 | 0.053390
 0.0350 | 0.711089 | 0.054526
 0.0450 | 0.457232 | 0.043077
 0.0550 | 0.409167 | 0.040495
 0.0650 | 0.349627 | 0.037603
 0.0750 | 0.259031 | 0.032658
 0.0850 | 0.277722 | 0.033246
 0.0950 | 0.137499 | 0.023636

This histogram shows the LL resummation of our starting dipole with the defined outside region. It presents the result in the form of $R(t)$ as given in (2.22) of [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660), with the parameter $t$ in the left column (center value of the bin), the Monte Carlo result of $R(t)$ in the middle and an error estimate for each bin on the right.

Other interesting values are the expansion parameters of the resummation as given in (4.3) of [doi:10.1007/JHEP09(2020)029](https://inspirehep.net/literature/1798660). The one-loop coefficient $\mathcal{S}^{(1)}$ can be extracted as 

In [16]:
round(shower.ngl1Loop,4)

-14.9399

and the two-loop one $\mathcal{S}^{(2)}$ consists of a non-global part directly extracted from the shower and an added global part, which corresponds to one  half  of  the  squared  one-loop  contribution:

In [17]:
round(shower.ngl2Loop+0.5*shower.ngl1Loop**2,4)

50.9485

Note that the format of the output as given in resummator\_dipole.py differs a little for simpler handling with multiple results collected from a cluster. 

Once finished playing around with this example, we are ready to discuss how to build Events from LHE files.